# Implementing the NFFT

In [ ]:
from __future__ import division

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

## 1. Do a straightforward implementation

In [ ]:
def adjoint_direct(x, f, N):
    k = -(N // 2) + np.arange(N)
    return np.dot(f, np.exp(2j * np.pi * k * x[:, None]))

In [ ]:
x = -0.5 + np.random.rand(1000)
f = np.sin(5 * 2 * np.pi * x) + np.cos(10 * 2 * np.pi * x)
plt.plot(x, f.real, '.k');

In [ ]:
N = 50
k = -(N // 2) + np.arange(N)
f_k = adjoint_direct(x, f, N)

plt.plot(k, f_k.real, k, f_k.imag)

## Easy version of the more complex implementation

In [ ]:
# equations C.1 from https://www-user.tu-chemnitz.de/~potts/paper/nfft3.pdf

def phi(x, n, m, sigma):
    b = (2 * sigma * m) / ((2 * sigma - 1) * np.pi)
    return np.exp(-(n * x) ** 2 / b) / np.sqrt(np.pi * b)


def phi_hat(k, n, m, sigma):
    b = (2 * sigma * m) / ((2 * sigma - 1) * np.pi)
    return np.exp(-b * (np.pi * k / n) ** 2) / n


def C(sigma, m):
    return 4 * np.exp(-m * np.pi * (1 - 1. / (2 * sigma - 1)))
        
    
def m_from_C(C, sigma):
    return int(np.ceil(-np.log(0.25 * C) / (np.pi * (1 - 1. / (2 * sigma - 1)))))


# Alg 3 from https://www-user.tu-chemnitz.de/~potts/paper/nfft3.pdf

def adjoint(x, f, N, m=None, sigma=5, tol=1E-8):
    n = N * sigma
    
    ell_grid = -(n // 2) + np.arange(n)
    x_grid = ell_grid / n
    
    if m is None:
        # TODO: determine this based on tol
        m = n // 2
    
    # TODO: this is the part that should be truncated using m
    shifted = lambda x: -0.5 + (x + 0.5) % 1
    g = np.dot(f, phi(shifted(x_grid - x[:, None]), n, m, sigma))
    
    # TODO: this sum should be computed via an FFT
    k = -(N // 2) + np.arange(N)
    ghat = np.dot(g, np.exp(2j * np.pi * k * x_grid[:, None]))
    
    return ghat / phi_hat(k, n, m, sigma) / (sigma * N)

In [ ]:
100 * C(5, 10.39)

In [ ]:
m_from_C(1E-10 / 100, 5)

In [ ]:
np.allclose(adjoint(x, f, N),
            adjoint_direct(x, f, N))

## Speeding up the FFT

In [ ]:
N = 100
xgrid = -0.5 + np.arange(N) / N
g = np.random.randn(N) + 1j * np.random.randn(N)
k = -(N // 2) + np.arange(N)

S1 = np.dot(g, np.exp(2j * np.pi * k * xgrid[:, None]))

S2 = N * np.fft.fftshift(np.fft.ifft(g)) * np.exp(2j * np.pi * 0.5 * k)

plt.plot(k, abs(S1), k, abs(S2))

np.allclose(S1, S2)

## Using the FFT in the Function

In [ ]:
def adjoint(x, f, N, m=None, sigma=5, tol=1E-8, use_fft=True):
    n = N * sigma
    
    ell_grid = -(n // 2) + np.arange(n)
    x_grid = ell_grid / n
    
    if m is None:
        # TODO: determine this based on tol
        m = n // 2
    
    # TODO: this is the part that should be truncated using m
    shifted = lambda x: -0.5 + (x + 0.5) % 1
    g = np.dot(f, phi(shifted(x_grid - x[:, None]), n, m, sigma))
    
    k = -(N // 2) + np.arange(N)
    if use_fft:
        ghat = n * np.fft.fftshift(np.fft.ifft(g))
        ghat = ghat[n // 2 - N // 2: n // 2 + N // 2]
        ghat *= np.exp(2j * np.pi * 0.5 * k)
    else:
        ghat = np.dot(g, np.exp(2j * np.pi * k * x_grid[:, None]))
    
    return ghat / phi_hat(k, n, m, sigma) / (sigma * N)

In [ ]:
x = -0.5 + np.random.rand(1000)
f = np.sin(5 * 2 * np.pi * x) + np.cos(10 * 2 * np.pi * x)

np.allclose(adjoint(x, f, N),
            adjoint_direct(x, f, N))

## Truncating the sum

In [ ]:
def g_matrix(x, N, sigma, m=None):
    n = N * sigma
    if m is None:
        m = n // 2
    assert m <= n // 2
    
    ell_grid = -(n // 2) + np.arange(n)
    x_grid = ell_grid / n
    
    shifted = lambda x: -0.5 + (x + 0.5) % 1
    return phi(shifted(x_grid - x[:, None]), n, m, sigma)

x = -0.5 + np.random.rand(50)

plt.imshow(g_matrix(x, 100, 1), aspect='auto')

We want to build this as a sparse matrix, with each row of width *2m*:

In [ ]:
from scipy.sparse import coo_matrix, csr_matrix

def g_matrix_sparse(x, N, sigma, m=None):
    n = N * sigma
    if m is None:
        m = n // 2
    assert m <= n // 2
    col_ind = np.floor(n * x[:, np.newaxis]).astype(int) + np.arange(-m, m)
    shifted = lambda x: -0.5 + (x + 0.5) % 1
    val = phi(shifted(x[:, None] - col_ind / n), n, m, sigma)
    col_ind += n // 2
    col_ind %= n
    
    indptr = np.arange(len(x) + 1) * col_ind.shape[1]
    return csr_matrix((val.ravel(), col_ind.ravel(), indptr), shape=(len(x), n))
    
    # row_ind = np.broadcast_to(np.arange(len(x))[:, None], col_ind.shape)
    # return coo_matrix((val.ravel(), (row_ind.ravel(), col_ind.ravel())), shape=(len(x), n))

m = 20

mat1 = g_matrix(x, 100, 1, m)
mat2 = g_matrix_sparse(x, 100, 1, m).toarray()

print(abs(mat1 - mat2).max())

np.allclose(mat1, mat2)

In [ ]:
def adjoint(x, f, N, sigma=5, m=None, tol=1E-8, use_fft=True, use_sparse=True):
    sigma = int(sigma)
    assert sigma >= 1
    
    N = int(N)
    n = N * sigma
    
    if m is None:
        m = m_from_C(tol / N, sigma)
    assert m <= n // 2
    print(m)
    
    # Compute the (truncated) sum
    shifted = lambda x: -0.5 + (x + 0.5) % 1
    if use_sparse:
        col_ind = np.floor(n * x[:, np.newaxis]).astype(int) + np.arange(-m, m)
        val = phi(shifted(x[:, None] - col_ind / n), n, m, sigma)
        col_ind += n // 2
        col_ind %= n

        indptr = np.arange(len(x) + 1) * col_ind.shape[1]
        mat = csr_matrix((val.ravel(), col_ind.ravel(), indptr), shape=(len(x), n))
        g = mat.T.dot(f)
    else:
        ell_grid = -(n // 2) + np.arange(n)
        x_grid = ell_grid / n
        g = np.dot(f, phi(shifted(x_grid - x[:, None]), n, m, sigma))
    
    k = -(N // 2) + np.arange(N)
    if use_fft:
        ghat = n * np.fft.fftshift(np.fft.ifft(g))
        ghat = ghat[n // 2 - N // 2: n // 2 + N // 2]
        ghat *= np.exp(2j * np.pi * 0.5 * k)
    else:
        ghat = np.dot(g, np.exp(2j * np.pi * k * x_grid[:, None]))
    
    return ghat / phi_hat(k, n, m, sigma) / (sigma * N)

In [ ]:
x = -0.5 + np.random.rand(1000)
f = np.sin(5 * 2 * np.pi * x) + np.cos(10 * 2 * np.pi * x)
N = 100

A1 = adjoint(x, f, N, tol=1E-8)
A2 = adjoint_direct(x, f, N)

print(np.max(abs(A2 - A1)))
np.allclose(A2, A1)